# nat-Zarcos Analysis Workflow -- v1


This notebook demonstrates the usage of the nat-zacros package for analyzing Zacros simulation data.

(Note: formerly named zacros_analysis_v1.ipynb)


## 0. Debug Automatic Versioning Snipits 

### find locations of sitepackages

In [ ]:
# import site; print(site.getsitepackages())
# import site; print(site.getusersitepackages())
import site
import os

site_packages = site.getsitepackages() + [site.getusersitepackages()]
for sp in site_packages:
    print(f"\nListing in: {sp}")
    if os.path.isdir(sp):
        for name in os.listdir(sp):
            if "nat_zacros" in name or "nat-zacros" in name or "egg-link" in name:
                print("  ", name)

In [ ]:
import importlib
import nat_zacros
importlib.reload(nat_zacros)
print(nat_zacros.__version__)

In [ ]:
import nat_zacros
print(nat_zacros.__file__)

In [ ]:
import os
print("CWD:", os.getcwd())
print("GIT DIR exists:", os.path.isdir(os.path.join(os.getcwd(), ".git")))

In [ ]:
import os
print("CWD:", os.getcwd())
print("GIT DIR exists:", os.path.isdir(os.path.join(os.getcwd(), ".git")))

In [ ]:
import setuptools_scm
print(setuptools_scm.get_version(root="c:/Users/a-DJA/GIT/nat_zacros"))

In [ ]:
import importlib
import nat_zacros
importlib.reload(nat_zacros)
print(nat_zacros.__version__)

## 1. Setup and Configuration

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle
from pathlib import Path
import platform
import subprocess
import time


# -----------------------------------------------------------------------------
# ------            This code no longer needed                           ------
# ------            use environment with nat-zarcos installed            ------
# -----------------------------------------------------------------------------
# # Add nat_zacros package to path (located in ../nat_zacros relative to this notebook)
# project_root = Path.cwd().parent  # Goes from O_Pt111 to GIT
# nat_zacros_pkg_dir = project_root / 'nat_zacros'
# if nat_zacros_pkg_dir.exists() and str(nat_zacros_pkg_dir) not in sys.path:
#     sys.path.insert(0, str(nat_zacros_pkg_dir))
#     print(f"Added to path: {nat_zacros_pkg_dir}")

import importlib
import nat_zacros
# importlib.reload(nat_zacros)

from nat_zacros import lattice, state, trajectory, load_trajectories_parallel

# Dictionary mapping usernames to data paths
user_paths = {
    'a-DJA'  : Path('c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations'),
    'akandra': Path('/home/akandra/O_Pt111/zacros_calculations'),
}

if platform.system() == 'Windows':
    username = os.getenv('USERNAME')
else:  # Linux/Mac
    username = os.getenv('USER')

# Get path for current user
if username in user_paths:
    dir_with_calculations = user_paths[username]
else:
    raise ValueError(f"Unknown user: {username}. Add your path to user_paths dictionary.")

print(f"Operating System  : {platform.system()}")
print(f"User              : {username}")
print(f"nat_zacros version: {nat_zacros.__version__}")
print(f"Data directory    : {dir_with_calculations.as_posix()}")

Operating System  : Windows
User              : a-DJA
nat_zacros version: 0.0.7.dev4+g63ceb25a0
Data directory    : c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations


## 2. Specify directories for a group of simulations

In [ ]:
# Set directories
log_dir = dir_with_calculations / 'fn_3leed'
data_dir = log_dir / 'jobs'
results_dir = log_dir / 'results'
log_file = data_dir / 'jobs.log'

# Create results directory
Path.mkdir(results_dir, exist_ok=True)

# Verify the file exists
if not log_file.exists():
    raise FileNotFoundError(f"Log file not found: {log_file}")

# Load json log file
with open(log_file, 'r') as f:
    log_header = f.readline().split()  # Read header
    log_entries = [json.loads(line) for line in f if line.strip()]

# Parse job information
run_dirs = [data_dir / str(entry[0]) for entry in log_entries]
temperatures = [entry[4] for entry in log_entries]
lat_size = [entry[2][0]*entry[2][1] for entry in log_entries]
n_ads = [entry[3][0] for entry in log_entries]
interactions = ['-'.join(entry[5][1:]) for entry in log_entries]
coverages = [n_ads[i]/lat_size[i] for i in range(len(n_ads))]
print(f"Found {len(run_dirs)} simulation runs")

## 3. Determine Equilibration Time

**Purpose:** Load full trajectories and plot energy vs time to determine equilibration.

**Workflow:** 
1. Load ALL trajectory data (no equilibration cutoff)
2. Compute average energy vs time
3. Plot and visually inspect to identify equilibration point
4. Use this to set the `fraction` parameter in the next section

In [ ]:
# Select run for analysis
run_number = 0  
rundir = run_dirs[run_number]
print(f"Analyzing run : {rundir.name}")
print(f"  Temperature : {temperatures[run_number]} K")
print(f"  Coverage    : {coverages[run_number]:.3f} ML")
print(f"  Interactions: {interactions[run_number]}")

# Find all trajectory directories
traj_dirs = sorted([d for d in rundir.iterdir() if d.is_dir() and d.name.startswith('traj_')])
print(f"Found {len(traj_dirs)} trajectories")

# Start timing
t_start = time.time()

print("\n" + "="*70)
print("Loading FULL trajectories with parallel I/O (all data, no cutoff)")
print("="*70)

# Create lattice (same for all trajectories)
lat = lattice(traj_dirs[0])

# Load ALL trajectory data (fraction=1.0 means no equilibration cutoff)
trajs_full = load_trajectories_parallel(lat, traj_dirs, fraction=1.0, n_workers=None)

t_elapsed = time.time() - t_start

print(f"\n{'='*70}")
print(f"Loading completed in {t_elapsed:.2f} seconds")
print(f"{'='*70}")
print(f"\nTrajectory info:")
print(f"  Number of trajectories: {len(trajs_full)}")
print(f"  States per trajectory: {len(trajs_full[0])} (full, not equilibrated)")
print(f"  Total states: {sum(len(t) for t in trajs_full)}")

# Extract energy data and plot
print("\n" + "="*70)
print("Computing average energy vs time from FULL trajectories")
print("="*70)

n_time_bins = 100

# Use minimum end time across all trajectories
end_time = min([traj.times[-1] for traj in trajs_full])
print(f"Common time range: 0 to {end_time:.2e} s")

# Create time bins starting from 0 (full trajectory)
time_bins = np.linspace(0, end_time, n_time_bins + 1)
time_bin_centers = 0.5 * (time_bins[:-1] + time_bins[1:])
energy_hist = np.zeros(n_time_bins)

# Accumulate energy from all FULL trajectories
for traj in trajs_full:
    times, energies = traj.get_energy_vs_time()
    
    # Bin the energy values
    for t, energy in zip(times, energies):
        if t <= end_time:
            bin_index = np.digitize(t, time_bins, right=False) - 1
            if 0 <= bin_index < n_time_bins:
                energy_hist[bin_index] += energy

# Average over trajectories
energy_hist /= len(trajs_full)

print(f"Binned energy data from {len(trajs_full)} trajectories")

# Skip first few bins (initialization artifacts)
skip_bins = 5

print(f"\nEnergy statistics (after skipping first {skip_bins} bins):")
print(f"  Mean:   {np.mean(energy_hist[skip_bins:]):.4f} eV")
print(f"  Std:    {np.std(energy_hist[skip_bins:]):.4f} eV")
print(f"  Min:    {np.min(energy_hist[skip_bins:]):.4f} eV")
print(f"  Max:    {np.max(energy_hist[skip_bins:]):.4f} eV")

# Plot energy vs time
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(time_bin_centers[skip_bins:], energy_hist[skip_bins:], 
        marker='o', linestyle='-', markersize=3, linewidth=1.5, color='blue')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Energy (eV)', fontsize=12)
ax.set_title(f'Energy vs Time - Full Trajectories\n{interactions[run_number]}, T={temperatures[run_number]} K, θ={coverages[run_number]:.3f}',
            fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estimate equilibration point
eq_fraction = 0.5
eq_bin = int(eq_fraction * n_time_bins)

print(f"\n{'='*70}")
print("INSPECT THE PLOT ABOVE TO DETERMINE EQUILIBRATION")
print("="*70)
print("\nVisual inspection guidance:")
print("  - Look for where energy stabilizes (becomes roughly constant)")
print("  - Initial transient = equilibration period")
print("  - Choose a fraction to discard (e.g., 0.5 = discard first 50%)")
print("\nTypical values:")
print("  fraction=0.3 → keep last 30% (long equilibration)")
print("  fraction=0.5 → keep last 50% (moderate equilibration)")
print("  fraction=0.7 → keep last 70% (short equilibration)")

print(f"\nEnergy statistics (after {eq_fraction*100:.0f}% equilibration):")
print(f"  Mean:   {np.mean(energy_hist[eq_bin:]):.4f} eV")
print(f"  Std:    {np.std(energy_hist[eq_bin:]):.4f} eV")
print(f"  Min:    {np.min(energy_hist[eq_bin:]):.4f} eV")
print(f"  Max:    {np.max(energy_hist[eq_bin:]):.4f} eV")
print("="*70)

## 4. Load Equilibrated Trajectories with Caching

This cell loads trajectories with equilibration cutoff and demonstrates caching:

1. **Check for cached data** (binary pickle file)
2. **If not cached**: Load using **parallel I/O** with equilibration fraction
3. **Save to cache** for future use

**Performance:**
- First run: ~6-10s (parallel loading)
- Subsequent runs: ~0.5s (cache loading)

**Important:** Set the `fraction` parameter based on the equilibration plot above.

In [ ]:
clear_cache = False   # Set to True to force reload

# Use same run_number as above
print(f"Loading equilibrated trajectories for run: {rundir.name}")

# Cache files for this run
cache_file = results_dir / f'{rundir.name}_trajs_eq.pkl'
gref_cache_file = results_dir / f'{rundir.name}_gref.pkl'

# Clear cache if requested
if clear_cache:
    if cache_file.exists():
        cache_file.unlink()
        print(f"Trajectory cache cleared: {cache_file.name}")
    if gref_cache_file.exists():
        gref_cache_file.unlink()
        print(f"g_ref cache cleared: {gref_cache_file.name}")

print(f"\nTrajectory cache: {cache_file.name}")
print(f"  Exists: {cache_file.exists()}")
print(f"g_ref cache: {gref_cache_file.name}")
print(f"  Exists: {gref_cache_file.exists()}")

# If cache exists, show its age and size
if cache_file.exists():
    import datetime
    mod_time = datetime.datetime.fromtimestamp(cache_file.stat().st_mtime)
    size_mb = cache_file.stat().st_size / 1024**2
    print(f"\nTrajectory cache info:")
    print(f"  Modified: {mod_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"  Size: {size_mb:.1f} MB")

# Start timing
t_start = time.time()

if not cache_file.exists():
    print("\n" + "="*70)
    print("FIRST RUN: Loading trajectories with parallel I/O")
    print("="*70)
    
    # Create lattice (already created above, but recreate for standalone execution)
    lat = lattice(traj_dirs[0])
    
    # Load trajectories in parallel with equilibration cutoff
    # ADJUST fraction based on equilibration plot above
    eq_fraction = 0.5  # Keep last 50% of trajectory
    print(f"Using equilibration fraction: {eq_fraction} (keeping last {eq_fraction*100:.0f}%)")
    trajs = load_trajectories_parallel(lat, traj_dirs, fraction=eq_fraction, n_workers=None)
    
    # Save trajectories to cache
    print(f"\nSaving trajectories to cache: {cache_file.name}")
    with open(cache_file, 'wb') as f:
        pickle.dump(trajs, f)
    print(f"Trajectory cache saved: {cache_file.stat().st_size / 1024**2:.1f} MB")
    
else:
    print("\n" + "="*70)
    print("CACHED: Loading trajectories from pickle file")
    print("="*70)
    
    # Load from cache
    with open(cache_file, 'rb') as f:
        trajs = pickle.load(f)
    print(f"Loaded {len(trajs)} trajectories from cache")

t_load = time.time() - t_start

# =========================================================================
# Compute and cache g_ref (reference RDF for normalization)
# =========================================================================
print("\n" + "="*70)
print("Computing reference RDF (g_ref) for lattice normalization")
print("="*70)

# RDF parameters (must match those used in Section 5)
r_max = 40.0  # Angstrom
dr = 0.1  # Bin width

if not gref_cache_file.exists():
    print("Computing g_ref (one-time calculation per lattice)...")
    t_gref_start = time.time()
    r_ref, g_ref = trajs[0].get_g_ref(r_max=r_max, dr=dr)
    t_gref = time.time() - t_gref_start
    
    print(f"g_ref computed in {t_gref:.2f} seconds")
    print(f"  Array size: {len(g_ref)} bins")
    print(f"  Distance range: 0 to {r_max:.1f} Å")
    
    # Save g_ref to cache
    print(f"\nSaving g_ref to cache: {gref_cache_file.name}")
    with open(gref_cache_file, 'wb') as f:
        pickle.dump((r_ref, g_ref), f)
    print(f"g_ref cache saved: {gref_cache_file.stat().st_size / 1024:.1f} KB")
else:
    print("Loading g_ref from cache...")
    t_gref_start = time.time()
    with open(gref_cache_file, 'rb') as f:
        r_ref, g_ref = pickle.load(f)
    t_gref = time.time() - t_gref_start
    print(f"g_ref loaded from cache in {t_gref:.2f} seconds")
    print(f"  Array size: {len(g_ref)} bins")

t_total = t_load + t_gref

print(f"\n{'='*70}")
print("LOADING SUMMARY")
print(f"{'='*70}")
print(f"Trajectories:  {t_load:6.2f} s")
print(f"g_ref:         {t_gref:6.2f} s")
print(f"Total:         {t_total:6.2f} s")
print(f"\nTrajectory info:")
print(f"  Number of trajectories: {len(trajs)}")
print(f"  States per trajectory: {len(trajs[0])} (equilibrated)")
print(f"  Total states: {sum(len(t) for t in trajs)}")

# Check if this looks like full or equilibrated data
if 'trajs_full' in locals():
    ratio = len(trajs[0]) / len(trajs_full[0])
    print(f"  Equilibration ratio: {ratio:.1%} of full trajectory")
    if ratio > 0.8:
        print(f"  ⚠️  WARNING: Cache may contain nearly full trajectories!")
        print(f"     Consider setting clear_cache=True and rerunning")

print("="*70)

## 5. Radial Distribution Function (RDF) Analysis

Compute RDF using **vectorized distance calculations** (OPTIMIZATION #1).

**Performance note:** For 10 trajectories, sequential computation with vectorization is fastest (~2s).  
Parallel RDF would be slower due to overhead.

In [ ]:
# RDF parameters (must match those in Section 4)
lattice_constant = 2.821135  # 1nn distance in Angstroms (Pt(111))
r_max = 40.0  # Angstrom
dr = 0.1  # Bin width

# g_ref was already computed and cached in Section 4
print("=" * 70)
print("Computing RDF for all trajectories")
print("=" * 70)
print(f"Using cached g_ref from Section 4")
print(f"Total states to process: {sum(len(t) for t in trajs)}")
print(f"States per trajectory: {len(trajs[0])}")

t_start = time.time()

# Compute RDF for each trajectory using vectorized distances
rdfs = []
for i, traj in enumerate(trajs):
    print(f"  Computing RDF {i+1}/{len(trajs)} ({len(traj)} states)...", end='\r')
    r, g = traj.get_rdf(r_max=r_max, dr=dr, g_ref=g_ref, vectorized=True)
    rdfs.append(g)

# Average RDFs across all trajectories
g_avg = np.mean(rdfs, axis=0)
g_std = np.std(rdfs, axis=0)

# Convert distance to units of lattice constant
r_a0 = r / lattice_constant

t_elapsed = time.time() - t_start

print(f"\n\n{'=' * 70}")
print("RDF COMPUTATION COMPLETE")
print("=" * 70)
print(f"Time elapsed: {t_elapsed:.2f} seconds")
print(f"  Trajectories: {len(trajs)}")
print(f"  Total states: {sum(len(t) for t in trajs)}")
print(f"  Average time per trajectory: {t_elapsed/len(trajs):.3f} s")
print(f"  Average time per state: {t_elapsed/sum(len(t) for t in trajs)*1000:.2f} ms")

# Find peak information
peak_idx = np.argmax(g_avg)
print(f"\nRDF Peak Analysis:")
print(f"  Peak position: {r_a0[peak_idx]:.2f} a₀ ({r[peak_idx]:.2f} Å)")
print(f"  Peak height: {g_avg[peak_idx]:.3f}")
print(f"  Note: No 1nn peak at 1.00 a₀ due to strong repulsive interaction")
print("=" * 70)

In [ ]:
# Plot RDF with error bars
fig, ax = plt.subplots(figsize=(10, 6))

# Calculate bar width based on bin spacing
bar_width = (dr / lattice_constant) * 1.5

# Plot bars with error bars
ax.bar(r_a0, g_avg, width=bar_width, 
       ec='blue', fc='lightblue', alpha=0.7,
       label=f'Average ({len(trajs)} traj)', align='center')
ax.errorbar(r_a0, g_avg, yerr=g_std, 
           fmt='none', ecolor='black', capsize=0, 
           linewidth=1, alpha=0.6, zorder=10)

# Reference line for ideal gas
ax.axhline(1.0, color='k', linestyle='--', linewidth=1.5, 
          alpha=0.5, label='Ideal gas (g=1)')

# Formatting
ax.set_xlabel(r'Distance ($r / a_0$)', fontsize=12)
ax.set_ylabel('g(r)', fontsize=12)
ax.set_title(f'Radial Distribution Function\n{interactions[0]}, T={temperatures[0]} K, θ={coverages[0]:.3f}',
            fontsize=13, fontweight='bold')
ax.legend(fontsize=10, loc='upper right')
ax.grid(True, alpha=0.3, axis='y')
ax.set_xlim(0, r_max/lattice_constant)
ax.set_ylim(0, max(g_avg) * 1.1)

plt.tight_layout()
plt.show()


## 6. Performance Summary

This notebook demonstrates the **recommended workflow** for Zacros analysis.

In [ ]:
print("="*70)
print("PERFORMANCE OPTIMIZATIONS SUMMARY")
print("="*70)
print()
print("✅ APPLIED OPTIMIZATIONS:")
print()
print("1. Vectorized Distance Calculations (50-100x speedup)")
print("   - Uses NumPy broadcasting for pairwise distances")
print("   - RDF computation: ~2s for 10 trajectories")
print("   - Enabled by: traj.get_rdf(vectorized=True)")
print()
print("2. Parallel Loading (5-10x speedup)")
print("   - Multiple trajectories loaded simultaneously")
print("   - Loading time: ~6-10s (vs ~60s sequential)")
print("   - Enabled by: load_trajectories_parallel()")
print()
print("3. Binary Caching (100x speedup for repeated analysis)")
print("   - Saves parsed trajectories to pickle file")
print("   - Subsequent loads: ~0.5s (vs ~60s parsing)")
print("   - Enabled by: pickle.dump() / pickle.load()")
print()
print("="*70)
print("TYPICAL PERFORMANCE (10 trajectories, 14 cores):")
print("="*70)
print()
print("First run (no cache):")
print("  Loading:        6-10 s  (parallel)")
print("  RDF computation: ~2 s   (vectorized sequential)")
print("  Total:          8-12 s")
print()
print("Subsequent runs (with cache):")
print("  Loading:        0.5 s   (pickle cache)")
print("  RDF computation: ~2 s   (vectorized sequential)")
print("  Total:          2.5 s")
print()
print("="*70)
print("NOT USED (counterproductive for typical datasets):")
print("="*70)
print()
print("❌ Parallel RDF computation")
print("   - Overhead (2-4s) exceeds computation time (2s)")
print("   - Only beneficial for >50 trajectories")
print("   - Sequential is FASTER for typical use")
print()
print("="*70)

## 7. Usage Notes

**Workflow summary:**
1. **Section 3**: Load full trajectories and plot energy to determine equilibration
2. **Section 4**: Load equilibrated trajectories (set `fraction` based on Section 3 plot)
3. **Section 5**: Perform RDF or other analyses on equilibrated data

**Best practices demonstrated:**
1. Always check for cached data before loading
2. Use parallel loading for I/O operations
3. Use vectorized distance calculations (default in get_rdf)
4. Use sequential loops for RDF computation (parallel has too much overhead)

**To analyze a different run:**
- Change `run_number` in Section 3
- The cache file will be automatically created for the new run

**To clear cache and force reload:**
```python
cache_file.unlink()  # Delete cache file
# Or set clear_cache = True in Section 4
```